## PART 6 - Concatenation des CV

In [2]:
# Chargement des librairies nécessaires
import pandas as pd
import spacy

# Chargement de la dataset
dataset = pd.read_csv("dataset_CV.csv", delimiter = ";")

# Instanciation de la bibliothèque fr_core_news_lg de Spacy
nlp = spacy.load("fr_core_news_lg")

# Tokenization des textes de CV
dataset["Tokenization_preparation"] = dataset["CV_text_brut"].apply(lambda x : str(x).replace("\n", " "))
dataset["Tokenization"] = dataset["Tokenization_preparation"].apply(lambda x : nlp(x))
dataset["Number_of_tokens"] = dataset["Tokenization"].apply(lambda x : len(x))

In [6]:
## Création d'une dataframe pour accueillir les features des CV
for i in range(len(dataset)):
    df = pd.DataFrame(columns = ['CV_Sentences'])

    ## Extraction du texte du CV pour appliquer la sentencisation des CV
    index = dataset.columns.get_loc("Tokenization_preparation")
    doc = dataset.iloc[i,index]                                                             ## CHANGER PAR UN i POUR BOUCLER SUR TOUS LES CV 
    doc = nlp(doc)

    ## Création d'une liste de phrases qui va accueillir le texte découpé en phrases grâce à la fonction doc.sents
    list_sentence = []
    for sent in doc.sents :
        list_sentence.append(sent)

    ## Affectation des phrases dans la colonne CV_Sentences et suppression des parenthèses
    df['CV_Sentences'] = list_sentence
    df['Sentences_CV_clean'] = df['CV_Sentences'].apply(lambda x : str(x).replace("(","").replace(")",""))

    ## Création de la colonne CV_number pour identifier les phrases de chaque CV
    for j in range(0,len(df)):
        index2 = dataset.columns.get_loc("CV_number")
        df['CV_Number'] = dataset.iloc[i,index2]                                              ## A MODIFIER j POUR LA GENERALISATION DE L'ALGO

    ## Création d'une colonne qui compte le nombre de phrase par CV
    df['Sentence_line'] = df.index

    ## Création d'une colonne qui compte le nombre de tokens dans la phrase
    df['Nb_tokens'] = df['CV_Sentences'].apply(lambda x : len(x))

    ## Création d'une colonne pour connaitre la position en pourcentage de la phrase par rapport au texte entier
    df['%texte_lu'] = 0
    token_count = 0
    for i in range(len(df)):
        token_count += df['Nb_tokens'][i]
        df['%texte_lu'][i] = round(((token_count / df['Nb_tokens'].sum()) * 100),2)         ## TROUVER COMMENT METTRE EN DECIMAL

    ## Création pour connaître le % de texte lu à partir de la fin de ligne
    df["%texte_lu_fin_ligne"] = 0                                                           ## REVOIR LE CODE POUR LE FAIRE FONCTIONNER
    nb_tokens_total = df["Nb_tokens"].sum()

    for k in range(len(df)):
        df["%texte_lu_fin_ligne"][k] += (nb_tokens_total - df['Nb_tokens'][k]) / nb_tokens_total

    ## Création d'une colonne pour savoir si le mot est alphanumerique ou pas
    df["Is_alpha"] = 0
    index3 = df.columns.get_loc("Sentences_CV_clean")
    
    for l in range(len(df)):
        doc = df.iloc[l,index3]
        doc = nlp(doc)
        list_token_is_alpha = []
        for token in doc:
            list_token_is_alpha.append(token.is_alpha)
        df["Is_alpha"][l] = list_token_is_alpha

    ## Création de la colonne Grammaire pour connaitre le type de mots de la phrase
    df["Grammar"] = 0
    index = df.columns.get_loc("Sentences_CV_clean")
    for m in range(len(df)):
        doc = df.iloc[m,index]
        doc = nlp(doc)
        list_token_pos = []
        for token in doc:
            list_token_pos.append(token.pos_)
            df["Grammar"][m] = list_token_pos

    ## Création de la colonne Label qui sera notre target. Soit 1 il faut supprimer soit 0 on doit garder
    df['Label'] = 0

    ## Création de la dataset avec les CV concatenés
    dataset_concat = pd.DataFrame()
    dataset_concat = dataset_concat.append(df, ignore_index=True)

## Création d'un fichier Excel avec toutes les phrases de tous les CV
PATH = "/Users/kinn/Desktop/Projet_BlindCV/data_CV.csv"
dataset_concat.to_csv(path_or_buf = PATH, index = True)

<ipython-input-6-3181aa971b00>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['%texte_lu'][i] = round(((token_count / df['Nb_tokens'].sum()) * 100),2)         ## TROUVER COMMENT METTRE EN DECIMAL
<ipython-input-6-3181aa971b00>:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["%texte_lu_fin_ligne"][k] += (nb_tokens_total - df['Nb_tokens'][k]) / nb_tokens_total
<ipython-input-6-3181aa971b00>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve